In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import tensorflow as tf
import numpy as np
import os
from PIL import Image
import time
from tqdm import tqdm

import sys
sys.path.append('..')
from input_pipeline import Pipeline

# Build a graph

In [ ]:
tf.reset_default_graph()

dataset_path = '/mnt/datasets/dan/moda/edanet/train/'
filenames = os.listdir(dataset_path)
filenames = [n for n in filenames if n.endswith('.tfrecords')]
filenames = [os.path.join(dataset_path, n) for n in sorted(filenames)]

params = {
    'batch_size': 16, 'num_labels': 13,
    'image_height': 256, 'image_width': 256
}

with tf.device('/cpu:0'):
    pipeline = Pipeline(filenames, is_training=True, params=params)
    dataset = pipeline.dataset
    iterator = tf.data.Iterator.from_structure(dataset.output_types, dataset.output_shapes)
    init = iterator.make_initializer(dataset)
    features, labels = iterator.get_next()

# Show an image

In [ ]:
color_by_label = {j: np.random.randint(0, 255, size=3, dtype='uint8') for j in range(13)}

def get_color_mask(sparse_mask):
    masks = []
    for j in range(13):
        m = (sparse_mask == (j + 1)).astype('uint8')
        masks.append(np.expand_dims(m, 2) * color_by_label[j])
        
    return np.stack(masks).max(0)

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    output_images, output_masks = sess.run([features, labels])

In [ ]:
i = 1
image = (255.0*output_images[i]).astype('uint8')
Image.fromarray(image)

# Show masks

In [ ]:
t = Image.fromarray(image)
t.putalpha(255)

mask = get_color_mask(output_masks[i])
m = Image.fromarray(mask)
m.putalpha(Image.fromarray(150*(mask > 0).any(2).astype('uint8')))
t.alpha_composite(m)
t

# Measure speed

In [ ]:
times = []
with tf.Session() as sess:
    sess.run(init)
    for _ in range(105):
        start = time.perf_counter()
        output = sess.run([features, labels])
        times.append(time.perf_counter() - start)

times = np.array(times[5:])
print(times.mean(), times.std())

# Get class weights

In [ ]:
tf.reset_default_graph()


dataset_path = '/mnt/datasets/dan/moda/edanet/train/'
filenames = os.listdir(dataset_path)
filenames = [n for n in filenames if n.endswith('.tfrecords')]
filenames = [os.path.join(dataset_path, n) for n in sorted(filenames)]


pipeline = Pipeline(filenames, is_training=False, params={'num_labels': 13})
dataset = pipeline.dataset
iterator = tf.data.Iterator.from_structure(dataset.output_types, dataset.output_shapes)
init = iterator.make_initializer(dataset)
_, labels = iterator.get_next()

In [ ]:
result = []
train_dataset_size = 42243

with tf.Session() as sess:
    sess.run(init)
    for _ in tqdm(range(train_dataset_size)):
        output = sess.run(labels)[0]
        h, w = output.shape
        label_count = np.bincount(output.reshape(-1), minlength=13 + 1)
        result.append((label_count, w, h))

In [ ]:
total_label_count = np.stack([x[0] for x in result], axis=0).sum(0)
total_area = sum(x[1] * x[2] for x in result)

In [ ]:
probability_of_label = total_label_count/total_area
# it has shape [num_labels + 1]

In [ ]:
k = 1.12
weights = 1.0/np.log(probability_of_label + k)
print(weights)

In [ ]:
np.save('class_weights.npy', weights.astype('float32'))